In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
aisles_df = pd.read_csv("aisles.csv")
departments_df = pd.read_csv("departments.csv")
order_products_prior_df = pd.read_csv("order_products__prior.csv")
order_products_train_df = pd.read_csv("order_products__train.csv")
orders_df = pd.read_csv("orders.csv")
products_df = pd.read_csv("products.csv")

In [3]:
aisles_df.describe()

,aisle_id
count,134.000000
mean,67.500000
std,38.826537
min,1.000000
25%,34.250000
50%,67.500000
75%,100.750000
max,134.000000


In [4]:
departments_df.describe()

,department_id
count,21.000000
mean,11.000000
std,6.204837
min,1.000000
25%,6.000000
50%,11.000000
75%,16.000000
max,21.000000


In [5]:
order_products_prior_df.describe()

,order_id,product_id,add_to_cart_order,reordered
count,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07
mean,1.710749e+06,2.557634e+04,8.351076e+00,5.896975e-01
std,9.873007e+05,1.409669e+04,7.126671e+00,4.918886e-01
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.559430e+05,1.353000e+04,3.000000e+00,0.000000e+00
50%,1.711048e+06,2.525600e+04,6.000000e+00,1.000000e+00
75%,2.565514e+06,3.793500e+04,1.100000e+01,1.000000e+00
max,3.421083e+06,4.968800e+04,1.450000e+02,1.000000e+00


In [6]:
order_products_train_df.describe()

,order_id,product_id,add_to_cart_order,reordered
count,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06
mean,1.706298e+06,2.555624e+04,8.758044e+00,5.985944e-01
std,9.897326e+05,1.412127e+04,7.423936e+00,4.901829e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.433700e+05,1.338000e+04,3.000000e+00,0.000000e+00
50%,1.701880e+06,2.529800e+04,7.000000e+00,1.000000e+00
75%,2.568023e+06,3.794000e+04,1.200000e+01,1.000000e+00
max,3.421070e+06,4.968800e+04,8.000000e+01,1.000000e+00


In [7]:
orders_df.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


In [8]:
products_df.describe()

,product_id,aisle_id,department_id
count,49688.000000,49688.000000,49688.000000
mean,24844.500000,67.769582,11.728687
std,14343.834425,38.316162,5.850410
min,1.000000,1.000000,1.000000
25%,12422.750000,35.000000,7.000000
50%,24844.500000,69.000000,13.000000
75%,37266.250000,100.000000,17.000000
max,49688.000000,134.000000,21.000000


In [9]:
aisles_df.shape

(134, 2)

In [10]:
departments_df.shape

(21, 2)

In [11]:
order_products_prior_df.shape # get product id to merge aisle, department, product

(32434489, 4)

In [12]:
order_products_train_df.shape

(1384617, 4)

In [13]:
orders_df.shape

(3421083, 7)

In [14]:
products_df.shape

(49688, 4)

In [15]:
base_df = order_products_train_df

In [16]:
base_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [17]:
base_df.shape

(1384617, 4)

In [18]:
order_counts = len(base_df.groupby('order_id')['order_id'].nunique())
order_counts

131209

In [19]:
product_value_counts = base_df['product_id'].value_counts()
product_value_counts

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
42744        1
5871         1
47237        1
9305         1
38900        1
Name: product_id, Length: 39123, dtype: int64

In [20]:
# This is a huge dataset so lets filter out any items that have not been ordered at
# least 1% of the time over the number of orders we have
product_list = product_value_counts[product_value_counts >= order_counts * .01].index.tolist()
filtered_df = base_df[base_df['product_id'].isin(product_list)]
filtered_df.shape

(319579, 4)

In [21]:
len(product_list)

104

In [22]:
products_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [23]:
products_df.shape

(49688, 4)

In [24]:
# Join in products dataset
df = filtered_df.set_index('product_id').join(products_df.set_index('product_id'))
df.reset_index(inplace=True)

In [25]:
df.drop(['add_to_cart_order', 'reordered', 'aisle_id', 'department_id'], axis=1, inplace=True)

In [26]:
df.head()

,product_id,order_id,product_name
0,196,915,Soda
1,196,3160,Soda
2,196,3785,Soda
3,196,5356,Soda
4,196,6187,Soda


In [27]:
df.shape

(319579, 3)

In [28]:
df.index

RangeIndex(start=0, stop=319579, step=1)

In [29]:
df.head()

,product_id,order_id,product_name
0,196,915,Soda
1,196,3160,Soda
2,196,3785,Soda
3,196,5356,Soda
4,196,6187,Soda


In [30]:
df_enc = df[['order_id', 'product_name']].pivot_table(index=['order_id'], columns=['product_name'], aggfunc=[len], fill_value=0)
df_enc

len                      \
product_name 100% Whole Wheat Bread 2% Reduced Fat Milk   
order_id                                                  
1                                 0                   0   
36                                0                   0   
38                                0                   0   
96                                0                   0   
98                                0                   0   
...                             ...                 ...   
3421026                           0                   0   
3421049                           0                   0   
3421056                           0                   0   
3421063                           0                   0   
3421070                           0                   0   

                                                                               \
product_name Apple Honeycrisp Organic Asparagus Bag of Organic Bananas Banana   
order_id                                                                        
1                                   0         0                      1      0   
36                                  0         1                      0      0   
38                                  0         0                      0      0   
96                                  0         0                      0      0   
98                                  0         0                      1      0   
...                               ...       ...                    ...    ...   
3421026                             0         0                      0      0   
3421049                             0         0                      0      0   
3421056                             0         0                      0      0   
3421063                             0         0                      0      0   
3421070                             0         0                      0      0   

                                                                           \
product_name Blueberries Boneless Skinless Chicken Breasts Broccoli Crown   
order_id                                                                    
1                      0                                 0              0   
36                     0                                 0              0   
38                     0                                 0              0   
96                     0                                 0              0   
98                     0                                 0              0   
...                  ...                               ...            ...   
3421026                0                                 0              0   
3421049                0                                 0              0   
3421056                0                                 0              0   
3421063                0                                 0              0   
3421070                0                                 0              0   

                               ...                                  \
product_name Bunched Cilantro  ... Sparkling Natural Mineral Water   
order_id                       ...                                   
1                           0  ...                               0   
36                          0  ...                               0   
38                          1  ...                               0   
96                          0  ...                               0   
98                          0  ...                               0   
...                       ...  ...                             ...   
3421026                     0  ...                               0   
3421049                     0  ...                               0   
3421056                     0  ...                               0   
3421063                     0  ...                               0   
3421070                     0  ...                               0   

          

In [31]:
def encode(x):
    if x >= 1:
        return 1
    # else
    return 0

In [32]:
df_enc = df_enc.applymap(encode)
df_enc

In [33]:
df_enc = df_enc[(df_enc[df_enc == 1]).sum(axis=1) > 1]
df_enc

len                      \
product_name 100% Whole Wheat Bread 2% Reduced Fat Milk   
order_id                                                  
1                                 0                   0   
36                                0                   0   
38                                0                   0   
96                                0                   0   
98                                0                   0   
...                             ...                 ...   
3420894                           0                   0   
3420895                           0                   0   
3420909                           0                   0   
3420996                           0                   0   
3420998                           0                   0   

                                                                               \
product_name Apple Honeycrisp Organic Asparagus Bag of Organic Bananas Banana   
order_id                                                                        
1                                   0         0                      1      0   
36                                  0         1                      0      0   
38                                  0         0                      0      0   
96                                  0         0                      0      0   
98                                  0         0                      1      0   
...                               ...       ...                    ...    ...   
3420894                             0         0                      1      0   
3420895                             0         0                      0      0   
3420909                             0         0                      0      0   
3420996                             0         0                      0      1   
3420998                             1         0                      0      0   

                                                                           \
product_name Blueberries Boneless Skinless Chicken Breasts Broccoli Crown   
order_id                                                                    
1                      0                                 0              0   
36                     0                                 0              0   
38                     0                                 0              0   
96                     0                                 0              0   
98                     0                                 0              0   
...                  ...                               ...            ...   
3420894                0                                 0              0   
3420895                0                                 0              0   
3420909                0                                 0              0   
3420996                0                                 0              0   
3420998                0                                 0              0   

                               ...                                  \
product_name Bunched Cilantro  ... Sparkling Natural Mineral Water   
order_id                       ...                                   
1                           0  ...                               0   
36                          0  ...                               0   
38                          1  ...                               0   
96                          0  ...                               0   
98                          0  ...                               0   
...                       ...  ...                             ...   
3420894                     0  ...                               0   
3420895                     0  ...                               0   
3420909                     0  ...                               0   
3420996                     0  ...                               0   
3420998                     0  ...                               0   

          

In [34]:
freq_itemsets = apriori(df=df_enc, min_support=0.01, use_colnames=True)

In [35]:
freq_itemsets.sort_values('support', ascending=False)

,support,itemsets
5,0.233308,"((len, Banana))"
4,0.201770,"((len, Bag of Organic Bananas))"
73,0.149713,"((len, Organic Strawberries))"
38,0.135134,"((len, Organic Baby Spinach))"
29,0.112976,"((len, Large Lemon))"
...,...,...
103,0.010148,"((len, Bag of Organic Bananas), (len, Apple Ho..."
174,0.010134,"((len, Organic Baby Spinach), (len, Organic Bl..."
171,0.010104,"((len, Organic Grape Tomatoes), (len, Organic ..."
150,0.010030,"((len, Banana), (len, Organic Yellow Onion))"


In [36]:
pd.set_option('display.max_rows', None)
association_rules(freq_itemsets, metric='lift', min_threshold=1).sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
91,"((len, Limes))","((len, Organic Cilantro))",0.084451,0.049811,0.014875,0.176141,3.536199,0.010669,1.153340
90,"((len, Organic Cilantro))","((len, Limes))",0.049811,0.084451,0.014875,0.298636,3.536199,0.010669,1.305383
130,"((len, Organic Garlic))","((len, Organic Yellow Onion))",0.059560,0.060890,0.012556,0.210813,3.462210,0.008930,1.189972
131,"((len, Organic Yellow Onion))","((len, Organic Garlic))",0.060890,0.059560,0.012556,0.206211,3.462210,0.008930,1.184747
74,"((len, Limes))","((len, Large Lemon))",0.084451,0.112976,0.023561,0.278992,2.469492,0.014020,1.230257
75,"((len, Large Lemon))","((len, Limes))",0.112976,0.084451,0.023561,0.208551,2.469492,0.014020,1.156801
135,"((len, Organic Hass Avocado))","((len, Organic Lemon))",0.101527,0.049353,0.011537,0.113633,2.302455,0.006526,1.072521
134,"((len, Organic Lemon))","((len, Organic Hass Avocado))",0.049353,0.101527,0.011537,0.233762,2.302455,0.006526,1.172577
155,"((len, Bag of Organic Bananas))","((len, Organic Strawberries), (len, Organic Ha...",0.201770,0.022734,0.010488,0.051980,2.286461,0.005901,1.030850
154,"((len, Organic Strawberries), (len, Organic Ha...","((len, Bag of Organic Bananas))",0.022734,0.201770,0.010488,0.461339,2.286461,0.005901,1.481878


In [37]:
pd.set_option('display.max_rows', 10)  # or 1000

# New Section